In [9]:
!pip install gradio langchain langchain_community langchain_milvus transformers pymilvus
!pip install sentence-transformers
!pip install google-generativeai langchain-google-genai
!pip install pypdf
!pip install pdfplumber
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyPDFLoader
import os
import gradio as gr
import tempfile
from langchain_milvus import Milvus
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.document_loaders import PDFPlumberLoader

In [10]:
#Steps to create Gemini API key in README.md
os.environ["GOOGLE_API_KEY"] = "<API_KEY>"

gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])

In [11]:
db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

In [12]:
filename = "IFCT.pdf"
loader = PDFPlumberLoader(filename)
documents = loader.load()

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=500,
    chunk_overlap=50
)

docs = splitter.split_documents(documents)

Token indices sequence length is longer than the specified maximum sequence length for this model (606 > 512). Running this sequence through the model will result in indexing errors


In [14]:
template = """User's prompt:{question}.You are a personalized nutrition assistant.The response should include meal plans, portion sizes with nutrient breakdowns, affordable local food options, medical-condition-specific do’s and don’ts, hydration, and lifestyle tips. All guidance must be personalized, culturally relevant, and practical to follow."""

prompt = PromptTemplate(template=template, input_variables=["question"])


In [15]:
llm_chain = LLMChain(llm=gemini_llm, prompt=prompt)
combine_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="question"
)

In [16]:
rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

In [17]:
def ask_NutritionAgent(query):
    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

In [18]:
iface = gr.Interface(
    fn=ask_NutritionAgent,
    inputs=gr.Textbox(
        label="Ask a question",
        placeholder="e.g.Give me a 2 day weight loss plan"
    ),
    outputs=gr.Textbox(label="Answer"),
    title="NutritionAgent",
    description="Ask doubts",
    theme="default"
)

In [19]:
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f4fc2d53106aa67c35.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
